## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-07-04-06-16 +0000,nyt,Suicide Bombing Kills 31 at Mosque in Pakistan...,https://www.nytimes.com/2026/02/06/world/asia/...
1,2026-02-07-04-00-00 +0000,wsj,Young Voters Can’t Get Enough of Japan’s Conse...,https://www.wsj.com/world/asia/young-voters-ca...
2,2026-02-07-04-00-00 +0000,wsj,"Squeezed by U.S. and China, the World’s Middle...",https://www.wsj.com/world/europe/squeezed-by-u...
3,2026-02-07-03-59-50 +0000,bbc,What we know about the abduction of Savannah G...,https://www.bbc.com/news/articles/cdex1nlrj07o...
4,2026-02-07-03-59-33 +0000,nyt,Authorities Investigate a New Message in Nancy...,https://www.nytimes.com/2026/02/06/us/savannah...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2469/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
77,trump,42
242,epstein,20
26,new,19
79,iran,14
78,talks,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
15,2026-02-07-03-07-00 +0000,wsj,A federal judge has ordered the Trump administ...,https://www.wsj.com/politics/policy/judge-bloc...,119
52,2026-02-07-01-00-00 +0000,wsj,President Trump chose a Federal Reserve chair ...,https://www.wsj.com/economy/central-banking/th...,91
16,2026-02-07-03-03-00 +0000,wsj,The Trump administration announced a new strat...,https://www.wsj.com/politics/national-security...,86
53,2026-02-07-00-52-00 +0000,wsj,President Trump posted a video including image...,https://www.wsj.com/politics/trump-obamas-apes...,85
18,2026-02-07-03-00-00 +0000,wsj,President Trump’s effort to push Republican st...,https://www.wsj.com/politics/elections/trumps-...,81


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
15,119,2026-02-07-03-07-00 +0000,wsj,A federal judge has ordered the Trump administ...,https://www.wsj.com/politics/policy/judge-bloc...
226,70,2026-02-06-17-16-34 +0000,nyt,Milan Winter Olympics Opening Ceremony: What T...,https://www.nytimes.com/2026/02/06/world/europ...
195,54,2026-02-06-18-31-15 +0000,nypost,Epstein suggested top lawyer Kathryn Ruemmler ...,https://nypost.com/2026/02/06/business/jeffrey...
146,52,2026-02-06-20-55-59 +0000,wapo,U.S. and Iran hold nuclear talks amid threats ...,https://www.washingtonpost.com/national-securi...
52,49,2026-02-07-01-00-00 +0000,wsj,President Trump chose a Federal Reserve chair ...,https://www.wsj.com/economy/central-banking/th...
145,45,2026-02-06-21-00-00 +0000,wsj,A White House full of babies cuts a striking i...,https://www.wsj.com/politics/white-house-famil...
245,43,2026-02-06-16-12-52 +0000,nypost,Ashleigh Banfield stands by single sourced acc...,https://nypost.com/2026/02/06/media/ashleigh-b...
273,43,2026-02-06-14-14-32 +0000,bbc,BBC at scene where Russian general was shot in...,https://www.bbc.com/news/videos/cx2y27je512o?a...
89,42,2026-02-06-23-05-00 +0000,bbc,Will Sunday's snap election gamble pay off for...,https://www.bbc.com/news/articles/ckgl53zll3jo...
123,38,2026-02-06-21-44-00 +0000,wsj,Bill and Hillary Clinton are once again prepar...,https://www.wsj.com/politics/looming-epstein-d...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
